# Predicting Temperature in London

![tower_bridge](tower_bridge.jpg)

As the climate changes, predicting the weather becomes ever more important for businesses. You have been asked to support on a machine learning project with the aim of building a pipeline to predict the climate in London, England. Specifically, the model should predict mean temperature in degrees Celsius (°C).

Since the weather depends on a lot of different factors, you will want to run a lot of experiments to determine what the best approach is to predict the weather. In this project, you will run experiments for different regression models predicting the mean temperature, using a combination of `sklearn` and `mlflow`.

You will be working with data stored in `london_weather.csv`, which contains the following columns:
- **date** - recorded date of measurement - (**int**)
- **cloud_cover** - cloud cover measurement in oktas - (**float**)
- **sunshine** - sunshine measurement in hours (hrs) - (**float**)
- **global_radiation** - irradiance measurement in Watt per square meter (W/m2) - (**float**)
- **max_temp** - maximum temperature recorded in degrees Celsius (°C) - (**float**)
- **mean_temp** - **target** mean temperature in degrees Celsius (°C) - (**float**)
- **min_temp** - minimum temperature recorded in degrees Celsius (°C) - (**float**)
- **precipitation** - precipitation measurement in millimeters (mm) - (**float**)
- **pressure** - pressure measurement in Pascals (Pa) - (**float**)
- **snow_depth** - snow depth measurement in centimeters (cm) - (**float**)

In [29]:
# Run this cell to install mlflow
!pip install mlflow

Defaulting to user installation because normal site-packages is not writeable


In [3]:
# Run this cell to import the modules you require
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [4]:
data = pd.read_csv('london_weather.csv')
data.head()

,date,cloud_cover,sunshine,global_radiation,max_temp,mean_temp,min_temp,precipitation,pressure,snow_depth
0,19790101,2.0,7.0,52.0,2.3,-4.1,-7.5,0.4,101900.0,9.0
1,19790102,6.0,1.7,27.0,1.6,-2.6,-7.5,0.0,102530.0,8.0
2,19790103,5.0,0.0,13.0,1.3,-2.8,-7.2,0.0,102050.0,4.0
3,19790104,8.0,0.0,13.0,-0.3,-2.6,-6.5,0.0,100840.0,2.0
4,19790105,6.0,2.0,29.0,5.6,-0.8,-1.4,0.0,102250.0,1.0


In [5]:
X = data.drop(columns=['mean_temp', 'date'])
y = data['mean_temp']

missing_y_indices = y.index[y.isnull()]
X = X.drop(index=missing_y_indices)
y = y.drop(index=missing_y_indices)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

In [40]:
def train_evaluate_model(X_train, X_test, y_train, y_test, model, model_name):
    with mlflow.start_run() as run:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        
        mlflow.log_params(model.get_params())
        mlflow.log_metric("rmse", rmse)
        
        mlflow.sklearn.log_model(model, model_name)
        
        print(f"Actual Test Data vs. Predicted Data by {model_name}:")
        print(pd.DataFrame({'Actual': y_test.values, 'Predicted': y_pred}))

In [41]:
linear_reg = LinearRegression()
decision_tree = DecisionTreeRegressor(random_state=42)
random_forest = RandomForestRegressor(n_estimators=100, random_state=42)

train_evaluate_model(X_train_scaled, X_test_scaled, y_train, y_test, linear_reg, "LinearRegression_Model")
train_evaluate_model(X_train_scaled, X_test_scaled, y_train, y_test, decision_tree, "DecisionTreeRegressor_Model")
train_evaluate_model(X_train_scaled, X_test_scaled, y_train, y_test, random_forest, "RandomForestRegressor_Model")

Actual Test Data vs. Predicted Data by LinearRegression_Model:
      Actual  Predicted
0       14.9  14.247566
1       22.7  21.948401
2        2.3   3.976799
3       14.8  14.340127
4       10.4   9.193783
...      ...        ...
3056    20.3  20.335549
3057    10.6  10.853203
3058     5.8   4.897430
3059    23.8  23.811295
3060    -0.4  -1.396940

[3061 rows x 2 columns]
Actual Test Data vs. Predicted Data by DecisionTreeRegressor_Model:
      Actual  Predicted
0       14.9       14.2
1       22.7       20.8
2        2.3        3.2
3       14.8       13.8
4       10.4        9.0
...      ...        ...
3056    20.3       20.0
3057    10.6       11.2
3058     5.8        5.4
3059    23.8       25.4
3060    -0.4       -0.2

[3061 rows x 2 columns]
Actual Test Data vs. Predicted Data by RandomForestRegressor_Model:
      Actual  Predicted
0       14.9     14.528
1       22.7     21.337
2        2.3      3.411
3       14.8     14.406
4       10.4      9.337
...      ...        ...
3056   

In [42]:
experiment_results = mlflow.search_runs()
experiment_results

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.rmse,params.n_estimators,params.max_samples,params.min_impurity_decrease,params.oob_score,params.min_samples_leaf,params.verbose,params.min_weight_fraction_leaf,params.criterion,params.random_state,params.n_jobs,params.min_samples_split,params.warm_start,params.bootstrap,params.max_features,params.ccp_alpha,params.max_depth,params.max_leaf_nodes,params.splitter,params.fit_intercept,params.copy_X,params.positive,tags.mlflow.runName,tags.mlflow.source.type,tags.mlflow.source.name,tags.mlflow.log-model.history,tags.mlflow.user
0,8f30feabe0584246980f52c91def1079,0,FINISHED,file:///work/files/workspace/mlruns/0/8f30feab...,2023-12-14 22:05:52.152000+00:00,2023-12-14 22:05:59.251000+00:00,0.853433,100,None,0.0,False,1,0,0.0,squared_error,42,None,2,False,True,1.0,0.0,None,None,None,None,None,None,lyrical-rook-800,LOCAL,/usr/lib/python3/dist-packages/python_kernel/k...,"[{""run_id"": ""8f30feabe0584246980f52c91def1079""...",repl
1,a5fa636015974e7597aa93d5b40b02b5,0,FINISHED,file:///work/files/workspace/mlruns/0/a5fa6360...,2023-12-14 22:05:49.573000+00:00,2023-12-14 22:05:52.128000+00:00,1.199342,None,None,0.0,None,1,None,0.0,squared_error,42,None,2,None,None,None,0.0,None,None,best,None,None,None,exultant-goose-13,LOCAL,/usr/lib/python3/dist-packages/python_kernel/k...,"[{""run_id"": ""a5fa636015974e7597aa93d5b40b02b5""...",repl
2,4e515a42e59b429c8a5924f74935e6a4,0,FINISHED,file:///work/files/workspace/mlruns/0/4e515a42...,2023-12-14 22:05:47.026000+00:00,2023-12-14 22:05:49.552000+00:00,0.873698,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,True,True,False,calm-bird-315,LOCAL,/usr/lib/python3/dist-packages/python_kernel/k...,"[{""run_id"": ""4e515a42e59b429c8a5924f74935e6a4""...",repl
3,bb95ac6ff31a4b5f938368e41cfc9849,0,FINISHED,file:///work/files/workspace/mlruns/0/bb95ac6f...,2023-12-14 21:35:31.350000+00:00,2023-12-14 21:35:38.584000+00:00,0.853433,100,None,0.0,False,1,0,0.0,squared_error,42,None,2,False,True,1.0,0.0,None,None,None,None,None,None,selective-rook-717,LOCAL,/usr/lib/python3/dist-packages/python_kernel/k...,"[{""run_id"": ""bb95ac6ff31a4b5f938368e41cfc9849""...",repl
4,c62b7f553b00433e8a28086cbf2e9c84,0,FINISHED,file:///work/files/workspace/mlruns/0/c62b7f55...,2023-12-14 21:35:28.748000+00:00,2023-12-14 21:35:31.332000+00:00,1.199342,None,None,0.0,None,1,None,0.0,squared_error,42,None,2,None,None,None,0.0,None,None,best,None,None,None,unique-cat-311,LOCAL,/usr/lib/python3/dist-packages/python_kernel/k...,"[{""run_id"": ""c62b7f553b00433e8a28086cbf2e9c84""...",repl
5,789b336af068414d8a750c84f4906fca,0,FINISHED,file:///work/files/workspace/mlruns/0/789b336a...,2023-12-14 21:35:25.404000+00:00,2023-12-14 21:35:28.729000+00:00,0.873698,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,True,True,False,delightful-carp-254,LOCAL,/usr/lib/python3/dist-packages/python_kernel/k...,"[{""run_id"": ""789b336af068414d8a750c84f4906fca""...",repl
6,8f5a6ae8632f46e497ced33b3eb06be7,0,FINISHED,file:///work/files/workspace/mlruns/0/8f5a6ae8...,2023-12-14 21:14:58.052000+00:00,2023-12-14 21:15:05.160000+00:00,0.853433,100,None,0.0,False,1,0,0.0,squared_error,42,None,2,False,True,1.0,0.0,None,None,None,None,None,None,masked-goat-937,LOCAL,/usr/lib/python3/dist-packages/python_kernel/k...,"[{""run_id"": ""8f5a6ae8632f46e497ced33b3eb06be7""...",repl
7,623e0eaa82ab481995fd01d39dd58098,0,FINISHED,file:///work/files/workspace/mlruns/0/623e0eaa...,2023-12-14 21:14:55.487000+00:00,2023-12-14 21:14:58.029000+00:00,1.199342,None,None,0.0,None,1,None,0.0,squared_error,42,None,2,None,None,None,0.0,None,None,best,None,None,None,adventurous-fox-191,LOCAL,/usr/lib/python3/dist-packages/python_kernel/k...,"[{""run_id"": ""623e0eaa82ab481995fd01d39dd58098""...",repl
8,f75277e778114c189137435b43dc60f3,0,FINISHED,file:///work/files/workspace/mlruns/0/f75277e7...,2023-12-14 21:14:52.948000+00:00,2023-12